##### Choose a day of significance to you (e.g., your birthday), and retrieve a 5% sample of the comments posted on this particular day across all 5 years of the dataset.

In [39]:
month = "2"
day = "1"

sampleSize = 0.05
seed = 5

#### Loading dataset

In [2]:
# from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import from_json, col
conf = SparkConf().setAppName('FirstSpark2').setMaster('Spark')
sc = SparkContext.getOrCreate()

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df = sqlContext.read.json("hdfs://orion11:20001/sample_sampled_reddit/")

print(type(df))
print(df.count())
df.show(1)

<class 'pyspark.sql.dataframe.DataFrame'>
30926243
+--------+--------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+---------+--------------+-------+------------+-----+-----+------------+--------+---------------+------------+---+------------+
|archived|        author|author_cakeday|author_flair_css_class|author_flair_text|                body|body_html|controversiality|created|created_utc|distinguished|downs|edited|gilded|     id|  link_id|mod_reports|name|parent_id|removal_reason|replies|retrieved_on|saved|score|score_hidden|stickied|      subreddit|subreddit_id|ups|user_reports|
+--------+--------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+---------+--------------+-------+--------

#### Finding Dataset

In [4]:
df2 = df.filter(
~(df.body.like('[deleted]'))
    & ~(df.body.isNull())
    & ~(df.author.like('[deleted]'))
    & ~(df.author.like('AutoModerator')) 
    & ~(df.author.rlike("[bB][oO][tT]"))

)

print(type(df2))
print(df2.count())
df2.show(1)

<class 'pyspark.sql.dataframe.DataFrame'>
27303462
+--------+--------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+---------+--------------+-------+------------+-----+-----+------------+--------+---------------+------------+---+------------+
|archived|        author|author_cakeday|author_flair_css_class|author_flair_text|                body|body_html|controversiality|created|created_utc|distinguished|downs|edited|gilded|     id|  link_id|mod_reports|name|parent_id|removal_reason|replies|retrieved_on|saved|score|score_hidden|stickied|      subreddit|subreddit_id|ups|user_reports|
+--------+--------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+---------+--------------+-------+--------

#### creating month and day column

In [24]:
from pyspark.sql.functions import year, month, dayofmonth, from_unixtime
from pyspark.sql.types import DateType

df3 = (df2
      .withColumn("day", dayofmonth(from_unixtime("created_utc").cast(DateType())))
      .withColumn("month",month(from_unixtime("created_utc").cast(DateType())))
      )

print(type(df3))
print(df3.count())
df3.show(1)

<class 'pyspark.sql.dataframe.DataFrame'>
27303462
+--------+--------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+---------+--------------+-------+------------+-----+-----+------------+--------+---------------+------------+---+------------+---+-----+
|archived|        author|author_cakeday|author_flair_css_class|author_flair_text|                body|body_html|controversiality|created|created_utc|distinguished|downs|edited|gilded|     id|  link_id|mod_reports|name|parent_id|removal_reason|replies|retrieved_on|saved|score|score_hidden|stickied|      subreddit|subreddit_id|ups|user_reports|day|month|
+--------+--------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+---------+-----------

####  dataset where month = month and day = day

In [33]:
df4 = df3.where((df3.day.like(day))&(df3.month.like(month)))
# df4 = df3.where(df3.day.like(day)).where(df3.month.like(month))


print(type(df4))
print(df4.count())
df4.show(1)

<class 'pyspark.sql.dataframe.DataFrame'>
85816
+--------+-----------+--------------+----------------------+-----------------+-------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+----------+--------------+-------+------------+-----+-----+------------+--------+----------+------------+---+------------+---+-----+
|archived|     author|author_cakeday|author_flair_css_class|author_flair_text|         body|body_html|controversiality|created|created_utc|distinguished|downs|edited|gilded|     id|  link_id|mod_reports|name| parent_id|removal_reason|replies|retrieved_on|saved|score|score_hidden|stickied| subreddit|subreddit_id|ups|user_reports|day|month|
+--------+-----------+--------------+----------------------+-----------------+-------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+----------+--------------+-------+------------+-----+-----+---

#### 5% sample of dataset

In [40]:
df5 = df4.sample(False, sampleSize, seed)

print(type(df5))
print(df5.count())
df5.show(1)

<class 'pyspark.sql.dataframe.DataFrame'>
4300
+--------+-------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+----------+--------------+-------+------------+-----+-----+------------+--------+--------------+------------+---+------------+---+-----+
|archived| author|author_cakeday|author_flair_css_class|author_flair_text|                body|body_html|controversiality|created|created_utc|distinguished|downs|edited|gilded|     id|  link_id|mod_reports|name| parent_id|removal_reason|replies|retrieved_on|saved|score|score_hidden|stickied|     subreddit|subreddit_id|ups|user_reports|day|month|
+--------+-------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+----------+--------------+-------+------------